In [ ]:
import os
import re
import pydicom
import numpy as np
from PIL import Image

def convert_and_rename_dicom_to_png(work_dir):
    """
    For each patient subfolder in work_dir:
      - Finds only unprefixed .dcm files
      - Sorts them by their Z-position
      - Renames them to 0001_<orig>.dcm, 0002_<orig>.dcm, …
      - Converts to 0001_<orig_base>.png (only if not already there)
    """
    for pid in sorted(os.listdir(work_dir)):
        pid_folder = os.path.join(work_dir, pid)
        if not os.path.isdir(pid_folder):
            continue

        slices = []
        for fname in os.listdir(pid_folder):
            if not fname.lower().endswith('.dcm'):
                continue
            # skip files that already have a 4-digit prefix
            if re.match(r'^\d{4}_', fname):
                continue

            full_path = os.path.join(pid_folder, fname)
            try:
                ds = pydicom.dcmread(full_path)
                z = float(ds.ImagePositionPatient[2])
                slices.append((z, fname, ds))
            except Exception as e:
                print(f"Skipping {full_path}: {e}")

        if not slices:
            print(f"Patient {pid}: no new DICOMs to process")
            continue

        # 2) Sort by Z
        slices.sort(key=lambda x: x[0])

        # 3) Rename + convert
        for idx, (_, fname, ds) in enumerate(slices, start=1):
            prefix = f"{idx:04d}_"
            old_dcm = os.path.join(pid_folder, fname)
            new_dcm = os.path.join(pid_folder, prefix + fname)

            # Rename the .dcm on disk
            os.rename(old_dcm, new_dcm)

            # Build PNG path
            base, _ = os.path.splitext(fname)
            png_name = f"{prefix}{base}.png"
            png_path = os.path.join(pid_folder, png_name)

            # Only convert if the PNG doesn't already exist
            if not os.path.exists(png_path):
                arr = ds.pixel_array.astype(np.float32)
                intercept = getattr(ds, 'RescaleIntercept', 0.0)
                slope     = getattr(ds, 'RescaleSlope',     1.0)
                arr = arr * slope + intercept

                pmin, pmax = arr.min(), arr.max()
                if pmax > pmin:
                    arr = (arr - pmin) / (pmax - pmin) * 255.0
                arr = arr.astype(np.uint8)

                Image.fromarray(arr).save(png_path)

        print(f"Patient {pid}: processed {len(slices)} file(s)")

WORK_DIR = './patient_images_0_1000'
convert_and_rename_dicom_to_png(WORK_DIR)

Patient ID_00526c11: processed 84 file(s)
Patient ID_00859e11: processed 32 file(s)
Patient ID_0086733c: processed 28 file(s)
Patient ID_00e30393: processed 31 file(s)
Patient ID_013cf2a2: processed 44 file(s)
Patient ID_02bdea77: processed 30 file(s)
Patient ID_03e3ed6d: processed 45 file(s)
Patient ID_0402b53e: processed 30 file(s)
Patient ID_046b584b: processed 141 file(s)
Patient ID_047b307a: processed 44 file(s)
Patient ID_048d5624: processed 34 file(s)
Patient ID_04a7b221: processed 33 file(s)
Patient ID_04fcd3da: processed 178 file(s)
Patient ID_0515ef2f: processed 28 file(s)
Patient ID_0531c34e: processed 34 file(s)
Patient ID_05bb1f88: processed 32 file(s)
Patient ID_05d3ffff: processed 42 file(s)
Patient ID_05fa237b: processed 65 file(s)
Patient ID_06185aa5: processed 34 file(s)
Patient ID_06b045bf: processed 247 file(s)
Patient ID_06c7d632: processed 28 file(s)
Patient ID_06edbec1: processed 34 file(s)
Patient ID_0762da12: processed 40 file(s)
Patient ID_07741131: processed 

In [ ]:
import os
import re
import pydicom
import numpy as np
from PIL import Image

# Three head-CT HU-windows
WINDOWS = {
    'brain':    {'center':  40,  'width':   80},
    'subdural': {'center':  80,  'width':  200},
    'bone':     {'center': 500,  'width': 2000},
}

def window_image(hu_array: np.ndarray, center: float, width: float) -> np.ndarray:
    """
    Apply a window to a HU array:
      - Clips to [center - width/2, center + width/2]
      - Linearly scales to [0,255]
    Returns uint8 image.
    """
    low  = center - width/2
    high = center + width/2
    clipped = np.clip(hu_array, low, high)
    scaled  = (clipped - low) / (high - low) * 255.0
    return scaled.astype(np.uint8)

def apply_windows_to_sorted_dataset(work_dir: str):
    """
    For each patient folder under work_dir, finds files named like '0001_*.dcm',
    converts to HU, applies each window, and writes out PNGs named
    '0001_base_brain.png', etc.
    """
    for pid in sorted(os.listdir(work_dir)):
        pid_folder = os.path.join(work_dir, pid)
        if not os.path.isdir(pid_folder):
            continue

        # find only the prefixed DICOMs
        dcm_files = sorted(f for f in os.listdir(pid_folder)
                           if re.match(r'^\d{4}_.*\.dcm$', f, flags=re.IGNORECASE))

        if not dcm_files:
            print(f"[{pid}] no sorted DICOMs found, skipping")
            continue

        for fname in dcm_files:
            dcm_path = os.path.join(pid_folder, fname)
            try:
                ds = pydicom.dcmread(dcm_path)
                arr = ds.pixel_array.astype(np.float32)
                intercept = getattr(ds, 'RescaleIntercept', 0.0)
                slope     = getattr(ds, 'RescaleSlope',     1.0)
                hu = arr * slope + intercept

                base, _ = os.path.splitext(fname)
                for win_name, win in WINDOWS.items():
                    img = window_image(hu, win['center'], win['width'])
                    out_name = f"{base}_{win_name}.png"
                    out_path = os.path.join(pid_folder, out_name)
                    if not os.path.exists(out_path):
                        Image.fromarray(img).save(out_path)

            except Exception as e:
                print(f"Failed on {dcm_path}: {e}")

        print(f"[{pid}] windows applied to {len(dcm_files)} slices")


WORK_DIR = './patient_images_0_1000' 
apply_windows_to_sorted_dataset(WORK_DIR)

[ID_00526c11] windows applied to 84 slices
[ID_00859e11] windows applied to 32 slices
[ID_0086733c] windows applied to 28 slices
[ID_00e30393] windows applied to 31 slices
[ID_013cf2a2] windows applied to 44 slices
[ID_02bdea77] windows applied to 30 slices
[ID_03e3ed6d] windows applied to 45 slices
[ID_0402b53e] windows applied to 30 slices
[ID_046b584b] windows applied to 141 slices
[ID_047b307a] windows applied to 44 slices
[ID_048d5624] windows applied to 34 slices
[ID_04a7b221] windows applied to 33 slices
[ID_04fcd3da] windows applied to 178 slices
[ID_0515ef2f] windows applied to 28 slices
[ID_0531c34e] windows applied to 34 slices
[ID_05bb1f88] windows applied to 32 slices
[ID_05d3ffff] windows applied to 42 slices
[ID_05fa237b] windows applied to 65 slices
[ID_06185aa5] windows applied to 34 slices
[ID_06b045bf] windows applied to 247 slices
[ID_06c7d632] windows applied to 28 slices
[ID_06edbec1] windows applied to 34 slices
[ID_0762da12] windows applied to 40 slices
[ID_0774